##Extrair informações de extratos de consórcios

In [ ]:
#Instalar biblioteca para manipular PDF
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 727.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.6 MB/s eta 0:00:00


In [112]:
import pdfplumber
import re
import pandas as pd

# Função para extrair o texto de todas as páginas do PDF
def extract_info_from_pdf(pdf_path):
    # Abrir o arquivo
    with pdfplumber.open(pdf_path) as pdf:
        # Inicializar uma variável para armazenar o texto de todas as páginas
        full_text = ""

        # Iterar sobre as páginas
        for page in pdf.pages:
            # Extrair o texto de cada página e concatenar com o texto anterior
            text = page.extract_text()
            full_text += text  # Acumula o texto de todas as páginas

        # Retornar o texto completo de todas as páginas
        return full_text

#Função para encontrar os dados no PDF usando expressões regulares
def extract_consorcio_info(text):
    info = {}

    #Encontrar nome da administradora
    match = re.search(r"^.*?LTDA", text)
    info['administradora'] = match.group().strip() if match else 'Não encontrado'

    #Encontrar número da cota
    match = re.search(r"Cota:\s*([0-9]+)", text)
    info['cota'] = match.group(1).strip() if match else 'Não encontrado'

    #Encontrar número do grupo
    match = re.search(r"Grupo:\s*([0-9\-]+)", text)
    info['grupo'] = match.group(1).strip() if match else 'Não encontrado'

    # Situação da cota
    match = re.search(r"Sit\. da Cota:\s*(\d{2}/\d{2}/\d{4}\s*(QUITADO|NORMAL))", text)
    info['situação_cota'] = match.group(1).strip() if match else 'Não encontrado'

    #Encontrar tipo de bem
    match = re.search(r"Bem básico:\s*(.+)", text)
    info['tipo_bem'] = match.group(1) if match else 'Não encontrado'

    #Encontrar taxa administrativa
    match = re.search(r"Taxa Adm:\s*([0-9,.]+)", text)
    info['taxa_adm'] = match.group(1) if match else 'Não encontrado'

    #Encontrar fundo de reserva
    match = re.search(r"Fdo\. Reserva:\s*([0-9,.]+)", text)
    info['fundo_reserva'] = match.group(1) if match else 'Não encontrado'

    #Encontrar data da contemplação
    match = re.search(r"Data da Contemplação:\s*([0-9/]+)", text)
    info['data_contemplação'] = match.group(1) if match else 'Não encontrado'

    #Encontrar tipo de contemplação
    match = re.search(r"Tipo de Contemplação:\s*(\w+)", text)
    info['tipo_contemplação'] = match.group(1) if match else 'Não encontrado'

    #Encontrar valor do crédito
    match = re.search(r"Valor do Crédito:\s*([0-9,.]+)", text)
    info['valor_crédito'] = match.group(1) if match else 'Não encontrado'


    return info


#Função para encontrar o último valor da parcela
def extract_valor_devido(text):

    #Localizar linhas que terminam com "PGTO PARC"
    parcelas = re.findall(r"(\d{3}\s\d{2}/\d{2}/\d{4}\s\d{2}/\d{2}/\d{4}\s\d{2}/\d{2}/\d{4}\s[\d,.]+\s[\d,.]+\s[\d,.]+\s[\d,.]+\s[\d,.]+\s[\d,.]+\s[\d,.]+\s[\d,.]+\s-?[\d,.]+\s[\d,.]+-0PGTO\sPARC)", text)

    #Se encontrar linhas com "PGTO PARC"
    if parcelas:
        #Pegar a última parcela com "PGTO PARC"
        ultima_parcela = parcelas[-1]

        #Dividir a última linha em colunas
        colunas = re.split(r"\s+", ultima_parcela)

        #Verificar se há pelo menos 7 colunas antes da coluna do "PGTO PARC"
        if len(colunas) >= 7:
            #O sétimo valor anterior ao "PGTO PARC" está na coluna 6
            setimo_valor_anterior = colunas[6]  #O valor da parcela está na 7ª coluna
            return setimo_valor_anterior
        else:
            return 'Menos de 7 colunas encontradas'
    else:
        return 'Nenhuma linha com "PGTO PARC" encontrada'

#Caminho para o PDF
pdf_path = "/content/Spengler 833 108 extrato de cota.pdf"


pdf_text = extract_info_from_pdf(pdf_path)
informacoes = extract_consorcio_info(pdf_text)


valor_devido = extract_valor_devido(pdf_text)

#Adicionar o valor da parcela ao dicionário de informações
informacoes['valor_devido_ultima_parcela'] = valor_devido

#Exibir as informações extraídas
for chave, valor in informacoes.items():
    print(f"{chave}: {valor}")


administradora: ADMINIST.DE CONSORCIO SPENGLER LTDA
cota: 0108
grupo: 000833
situação_cota: 22/12/2023 QUITADO
tipo_bem: 0346 CREDITO R/A CR 46 56.442,70
taxa_adm: 16,00
fundo_reserva: 1,00
data_contemplação: 24/04/2023
tipo_contemplação: Lance
valor_crédito: 56.442,70
valor_devido_ultima_parcela: 968,84


In [113]:
import pandas as pd
#Criar o DataFrame com as informações extraídas do PDF
df = pd.DataFrame([informacoes])


display(df)

,administradora,cota,grupo,situação_cota,tipo_bem,taxa_adm,fundo_reserva,data_contemplação,tipo_contemplação,valor_crédito,valor_devido_ultima_parcela
0,ADMINIST.DE CONSORCIO SPENGLER LTDA,0108,000833,22/12/2023 QUITADO,"0346 CREDITO R/A CR 46 56.442,70","16,00","1,00",24/04/2023,Lance,"56.442,70","968,84"


#Colocando mais de um arquivo no dataframe

In [114]:
pdf_path = []

# oletar os caminhos de cada PDF
while True:
    entrada = input('Digite o caminho do PDF ou aperte ENTER para sair: ')
    if entrada == '':
        break
    pdf_path.append(entrada)

#Lista para armazenar as informações de todos os PDFs
todas_informacoes = []

#Iterar sobre cada caminho de PDF
for caminho in pdf_path:
    pdf_text = extract_info_from_pdf(caminho)
    informacoes = extract_consorcio_info(pdf_text)


    valor_devido = extract_valor_devido(pdf_text)

    informacoes['valor_devido_ultima_parcela'] = valor_devido

    todas_informacoes.append(informacoes)

#Criar o DataFrame com as informações extraídas de todos os PDFs
df = pd.DataFrame(todas_informacoes)

#Exibir o DataFrame
display(df)

Digite o caminho do PDF ou aperte ENTER para sair: /content/Sinosserra 1041 314 Extrato.PDF
Digite o caminho do PDF ou aperte ENTER para sair: /content/Sinosserra 5235 33 Extrato (1).PDF
Digite o caminho do PDF ou aperte ENTER para sair: /content/Spengler 833 108 extrato de cota.pdf
Digite o caminho do PDF ou aperte ENTER para sair: /content/Sponchiado 1290-194 extrato.PDF
Digite o caminho do PDF ou aperte ENTER para sair: 


,administradora,cota,grupo,situação_cota,tipo_bem,taxa_adm,fundo_reserva,data_contemplação,tipo_contemplação,valor_crédito,valor_devido_ultima_parcela
0,SINOSSERRA ADM. DE CONSÓRCIOS LTDA,0314,001041,04/08/2022 NORMAL,"1150 IMOVEL RESIDENCIAL 100.000,00 158.704,92","27,00","0,50",Não encontrado,Valor,"0,00","1.013,75"
1,SINOSSERRA ADM. DE CONSÓRCIOS LTDA,0033,005235,05/07/2022 QUITADO,"2076 ONIX JOY RFA 65% 48.918,40","16,00","1,00",26/01/2022,Lance,"42.380,00","291,28"
2,ADMINIST.DE CONSORCIO SPENGLER LTDA,0108,000833,22/12/2023 QUITADO,"0346 CREDITO R/A CR 46 56.442,70","16,00","1,00",24/04/2023,Lance,"56.442,70","968,84"
3,SPONCHIADO ADM. DE CONSORCIOS LTDA,0194,001290,26/02/2024 QUITADO,"9674 Automóvel 17.500,00 - Gr.1290 28.305,45","14,00","0,00",20/02/2024,Lance,"28.305,45","448,13"


In [115]:
#exportar para o excel
df.to_excel('consorcio.xlsx', index=False)